In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

### Training

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import Trainer

trainer = Trainer(
    model, 
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [6]:
trainer.train()

C:\_MAIN\Workspace\Project\env\nlp-study\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
500,0.620800
1000,0.563100


TrainOutput(global_step=1377, training_loss=0.5552967988501101, metrics={'train_runtime': 69.8214, 'train_samples_per_second': 157.602, 'train_steps_per_second': 19.722, 'total_flos': 405114969714960.0, 'train_loss': 0.5552967988501101, 'epoch': 3.0})

### Evaluation

In [7]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [14]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [16]:
from datasets import load_metric

metric = load_metric("glue", "mrpc", trust_remote_code=True)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.803921568627451, 'f1': 0.8648648648648649}

In [17]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc", trust_remote_code=True)
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.450852,0.786765,0.862124
2,0.557300,0.501723,0.850490,0.894281
3,0.321500,0.627001,0.855392,0.898100


TrainOutput(global_step=1377, training_loss=0.36769581048836636, metrics={'train_runtime': 73.7696, 'train_samples_per_second': 149.167, 'train_steps_per_second': 18.666, 'total_flos': 405114969714960.0, 'train_loss': 0.36769581048836636, 'epoch': 3.0})